In [1]:
import findspark

In [2]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
pyspark

<module 'pyspark' from '/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/__init__.py'>

In [5]:
sc = pyspark.SparkContext()

In [6]:
from pyspark.sql import *

In [7]:
spark = SparkSession \
    .builder \
    .appName("Movie review") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [8]:
#dataframe = spark.read.csv('/home/cse587/train.csv', inferSchema=True)

In [9]:
df = spark.read.load("/home/cse587/train.csv", format='csv', inferSchema="true",header="true", escape = '"', seperator = ",")

In [10]:
#df.printSchema()

In [11]:
#df.head(5)

In [12]:
df.columns
df.select("genre").show()

+--------------------+
|               genre|
+--------------------+
|['World cinema', ...|
|['Action/Adventur...|
|['Musical', 'Acti...|
|          ['Comedy']|
|['Crime Fiction',...|
|['Action/Adventur...|
|['Thriller', 'Dra...|
|           ['Drama']|
|['Black-and-white...|
|['Animation', 'Sh...|
|          ['Comedy']|
|['Crime Fiction',...|
|          ['Comedy']|
|          ['Comedy']|
|          ['Horror']|
|['Crime Fiction',...|
|           ['Drama']|
|['Crime Fiction',...|
|  ['Indie', 'Drama']|
|           ['Drama']|
+--------------------+
only showing top 20 rows



In [13]:
#text_split = text_file.flatMap(lambda line: line.split(" "))

#text_word = text_split.map(lambda word: (word,1))

#word_count = text_word.reduceByKey(lambda a,b: a + b)

In [14]:
newdf = df.union(df)
newdf.count()

62222

In [15]:
df.describe('movie_id').show()

+-------+--------------------+
|summary|            movie_id|
+-------+--------------------+
|  count|               31111|
|   mean|1.4278167534139128E7|
| stddev|1.0880709619946605E7|
|    min|            10000053|
|    max|             9999280|
+-------+--------------------+



In [16]:
#word_count.collect()

In [17]:
df.groupby('genre').count().show()

+--------------------+-----+
|               genre|count|
+--------------------+-----+
|    ['Romance Film']|  362|
|['Crime Fiction',...|    1|
|['Crime Fiction',...|    1|
|['Adventure', 'Co...|    1|
|['Crime Fiction',...|    2|
|['Crime Fiction',...|    1|
|['Musical', 'Indi...|    1|
|['Action/Adventur...|    1|
|['World cinema', ...|    6|
|['Musical', 'Acti...|    2|
|['Romantic comedy...|    1|
|['Black-and-white...|    1|
|['Crime Fiction',...|    5|
|['Crime Fiction',...|    1|
|['Musical', 'Acti...|    1|
|['Thriller', 'Mys...|    1|
|['Action', 'Comed...|    1|
|['Action', 'Comed...|    2|
|['Comedy', 'Roman...|    1|
|['Action/Adventur...|    1|
+--------------------+-----+
only showing top 20 rows



In [18]:
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/cse587/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Relate movie_id and movie_name
movie_name = {}
labels = set()
for row in df.collect():
    if row['movie_id'] != None:
        movie_name[row['movie_id']] = row['movie_name']
    if row['genre'] != None:
        elements = row['genre'].split(',')
        for genre in elements:
            genre = genre.replace('[','')
            genre = genre.replace(']','')
            genre = genre.replace("\'",'')
            genre = genre.replace(" ",'')
            labels.add(genre)

    
labels

#print(len(labels))
#print(labels)
#print(movie_name)

{'Action',
 'Action/Adventure',
 'Adventure',
 'Animation',
 'Black-and-white',
 'Comedy',
 'CrimeFiction',
 'Drama',
 'FamilyFilm',
 'Horror',
 'Indie',
 'Musical',
 'Mystery',
 'RomanceFilm',
 'Romanticcomedy',
 'Romanticdrama',
 'ScienceFiction',
 'ShortFilm',
 'Thriller',
 'Worldcinema'}

In [83]:
from collections import defaultdict
word_frequency = defaultdict(int)
for row in df.collect():
    if row['plot'] != None:
        line = row['plot'].strip()
        line = line.lower()
        line = line.replace('_','')
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        words = tokenizer.tokenize(line)
        for word in words:
            word = re.sub(r'\d+', '', word)
            # Remove stopwords and invalid words. 
            if word not in stop_words and word != "":
                  word_frequency[word] = word_frequency[word] + 1
word_frequency = sorted(word_frequency.items(), key=lambda kv: kv[1], reverse = True)

word_frequency = [v[0] for v in word_frequency[2000:4000]]
key_words = set(word_frequency)
#print(key_words)

In [84]:
#print(labels)

In [85]:
label_set = [[0]* len(labels) for i in range(len(movie_name))]
labels_list = list(labels)
i = 0
for row in df.collect():
    if row['genre'] != None:
            elements = row['genre'].split(',')
            for genre in elements:
                genre = genre.replace('[','')
                genre = genre.replace(']','')
                genre = genre.replace("\'",'')
                genre = genre.replace(" ",'')
                for label in range (len(labels_list)):
                    if genre == labels_list[label]:
                        label_set[i][label] = 1
    i = i + 1

In [86]:
dataset = [[0]* len(key_words) for i in range(len(movie_name))]

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

In [88]:

i = 0
#print(len(movie_name))
for row in df.collect():
    j = 0
    line = row['plot']
    if line != None:
        line = line.lower()
        line = line.replace('_','')
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        words = tokenizer.tokenize(line)
        for word in key_words:
            if word in words:
                dataset[i][j] = words.count(word)
            j = j + 1
    i = i + 1

In [89]:
#word_count.saveAsTextFile('/home/cse587/wordcount')


In [90]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [110]:
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
X_train = np.array(dataset)
X_scaled = preprocessing.scale(X_train)
Y_train = np.array(label_set)

X_scaled, X_val, Y_train, Y_val = train_test_split(
X_scaled, Y_train, test_size=0.2)
print(len(X_scaled))
print(X_val)
print(len(Y_train))
print(Y_val)

24888
[[-0.10955259 -0.02953313 -0.08169301 ... -0.08233821 -0.08051411
  -0.09019731]
 [-0.10955259 -0.02953313 -0.08169301 ... -0.08233821 -0.08051411
  -0.09019731]
 [-0.10955259 -0.02953313 -0.08169301 ... -0.08233821 -0.08051411
  -0.09019731]
 ...
 [-0.10955259 -0.02953313 -0.08169301 ... -0.08233821 -0.08051411
  -0.09019731]
 [-0.10955259 -0.02953313 -0.08169301 ... -0.08233821 -0.08051411
  -0.09019731]
 [-0.10955259 -0.02953313 -0.08169301 ... -0.08233821 -0.08051411
  -0.09019731]]
24888
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [103]:
clf.fit(X_scaled, Y_train)

/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/cse587/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [111]:
y_pred = clf.predict(X_val)

In [112]:
#y_pred[1]
#labels

In [113]:
def get_labels(label, y_pred):
    labels = []
    for i in range (len(y_pred)):
        if y_pred[i] == 1:
            labels.append(label[i])
    return labels
for i in range (len(y_pred)):
    print(i+1,get_labels(list(labels), y_pred[i]))

1 ['Comedy', 'ShortFilm']
2 ['Action/Adventure', 'Action']
3 ['Horror', 'Mystery']
4 ['Horror', 'Comedy', 'Action/Adventure', 'Action']
5 ['Drama']
6 ['Horror', 'Comedy']
7 []
8 ['Comedy']
9 ['Drama', 'Musical']
10 ['Drama']
11 ['Drama']
12 ['Comedy', 'Drama']
13 ['Comedy', 'Drama', 'Worldcinema']
14 ['Romanticcomedy', 'Comedy', 'RomanceFilm']
15 ['Action', 'Drama']
16 []
17 ['Horror', 'ScienceFiction']
18 ['Drama']
19 ['Drama', 'ShortFilm']
20 []
21 []
22 ['RomanceFilm', 'Drama', 'Romanticdrama']
23 ['ScienceFiction']
24 ['Action', 'Thriller', 'CrimeFiction']
25 ['Action/Adventure', 'Drama', 'CrimeFiction']
26 ['Adventure', 'ScienceFiction']
27 ['Comedy']
28 ['Romanticcomedy', 'Drama', 'Worldcinema', 'Musical']
29 []
30 []
31 []
32 ['FamilyFilm', 'Comedy']
33 ['Mystery', 'Thriller', 'Drama', 'CrimeFiction']
34 ['FamilyFilm', 'Animation', 'ShortFilm']
35 ['Drama']
36 ['Action/Adventure', 'Action', 'Drama']
37 ['Action/Adventure', 'Action', 'Drama']
38 ['FamilyFilm', 'Drama', 'Musical']

1081 ['Drama', 'Musical']
1082 ['Horror']
1083 ['Action']
1084 ['FamilyFilm', 'Comedy', 'Animation', 'ShortFilm']
1085 ['RomanceFilm', 'Romanticdrama']
1086 ['Comedy', 'Action/Adventure', 'Action', 'Thriller', 'CrimeFiction']
1087 ['Horror', 'Thriller', 'Drama']
1088 ['Drama']
1089 []
1090 ['FamilyFilm', 'Comedy', 'Animation', 'ShortFilm']
1091 ['Drama']
1092 []
1093 ['Black-and-white', 'Animation', 'RomanceFilm', 'Drama', 'Romanticdrama']
1094 []
1095 []
1096 ['Drama']
1097 ['Action/Adventure', 'Worldcinema', 'ScienceFiction']
1098 ['Action/Adventure', 'Drama']
1099 []
1100 ['Comedy']
1101 ['Action', 'Thriller', 'Drama', 'CrimeFiction', 'Adventure']
1102 []
1103 ['Horror', 'Action', 'Adventure', 'ScienceFiction']
1104 ['Black-and-white', 'Comedy', 'ShortFilm']
1105 ['Action/Adventure', 'RomanceFilm', 'Action', 'Drama', 'Adventure']
1106 ['Indie', 'RomanceFilm']
1107 ['Comedy', 'Drama']
1108 ['Drama']
1109 ['Animation']
1110 []
1111 ['Drama']
1112 []
1113 ['Drama']
1114 []
1115 ['Anima

2581 ['Comedy', 'RomanceFilm', 'Drama']
2582 []
2583 ['Drama']
2584 []
2585 ['Romanticcomedy', 'Comedy', 'Drama']
2586 ['Drama']
2587 ['Black-and-white', 'Comedy', 'ShortFilm']
2588 ['ShortFilm']
2589 []
2590 ['FamilyFilm', 'Animation', 'ShortFilm']
2591 ['Horror']
2592 ['Comedy', 'Worldcinema']
2593 ['Black-and-white', 'Drama']
2594 ['Comedy', 'Drama']
2595 ['Drama']
2596 ['Drama']
2597 ['Comedy', 'Drama']
2598 ['Action', 'Adventure']
2599 ['Drama']
2600 ['Drama']
2601 ['FamilyFilm', 'Adventure']
2602 []
2603 ['RomanceFilm', 'Drama', 'Romanticdrama']
2604 ['Horror', 'ScienceFiction']
2605 ['Drama']
2606 []
2607 ['Drama']
2608 ['Comedy']
2609 ['Comedy']
2610 []
2611 []
2612 ['Drama']
2613 ['Drama']
2614 ['Drama']
2615 ['Mystery']
2616 ['Drama']
2617 ['Animation', 'Drama', 'ShortFilm']
2618 ['Comedy']
2619 ['Comedy', 'RomanceFilm']
2620 []
2621 []
2622 ['Comedy']
2623 ['Drama']
2624 ['Drama']
2625 []
2626 ['Drama']
2627 ['Comedy', 'RomanceFilm', 'Drama', 'Romanticdrama']
2628 []
2629 ['

4581 []
4582 ['Drama']
4583 []
4584 ['Horror', 'Thriller', 'Drama']
4585 []
4586 ['Drama']
4587 ['Drama']
4588 ['Comedy', 'Drama']
4589 []
4590 ['Comedy']
4591 ['Horror']
4592 ['Action', 'Drama']
4593 ['Romanticcomedy', 'Comedy', 'RomanceFilm', 'Drama', 'Musical', 'Romanticdrama']
4594 ['Drama']
4595 ['Thriller', 'Drama']
4596 []
4597 ['Indie', 'Comedy', 'Drama', 'CrimeFiction']
4598 ['Mystery', 'Indie', 'Drama']
4599 ['Drama']
4600 ['Drama']
4601 ['Drama']
4602 ['FamilyFilm']
4603 []
4604 ['Drama', 'Worldcinema']
4605 ['Drama']
4606 ['Comedy', 'Drama']
4607 ['Horror', 'Mystery', 'Thriller']
4608 ['FamilyFilm', 'Black-and-white', 'Indie', 'Comedy', 'ShortFilm']
4609 ['FamilyFilm', 'Comedy', 'Drama']
4610 []
4611 ['RomanceFilm', 'Drama']
4612 ['Drama']
4613 ['Drama']
4614 ['Indie', 'Comedy', 'Drama']
4615 ['Mystery']
4616 []
4617 ['Drama', 'Worldcinema']
4618 ['Comedy', 'Action']
4619 ['Comedy', 'Action/Adventure', 'Action']
4620 ['Animation']
4621 ['Black-and-white', 'Drama']
4622 ['Dr

In [117]:
print(Y_val)
print(y_pred)
f1_scores = f1_score(Y_val,y_pred)
f1_scores

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


TypeError: 'numpy.float64' object is not callable

0.022017935778342067
